# Notebook used for dev of instrumentation setup for class NevergradOptimizer 

In [1]:
from brian2 import *
from brian2tools import *
from nevergrad.optimization import optimizerlib, registry
from nevergrad import instrumentation as inst


WARNING    /home/alteska/anaconda3/envs/brian_opt/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
 [py.warnings]


## Dev Passing the Arguments to creat bounds 

In [2]:
parameter_names = {'E', 'g'}
bounds = [[-5, 5], [0, 10]]

In [3]:
method='DE'

In [4]:
arg1 = inst.var.Array(1).bounded(-5, 5).asscalar()
arg2 = inst.var.Array(1).bounded(0, 10).asscalar()
instrum = inst.Instrumentation(arg1, arg2)
optim = optimizerlib.registry[method](instrumentation=instrum, budget=10000)


In [5]:
instruments = []

for i, name in enumerate(parameter_names):
    print(name, bounds[i])
    vars()[name] = inst.var.Array(1).bounded(*bounds[i]).asscalar()
    instruments.append(vars()[name])



g [-5, 5]
E [0, 10]


In [6]:
instruments

[Array(shape=(1,), transforms=[ArctanBound(a_max=5, a_min=-5)]),
 Array(shape=(1,), transforms=[ArctanBound(a_max=10, a_min=0)])]

In [7]:
instrum = inst.Instrumentation(*instruments)

In [8]:
optim = optimizerlib.registry[method](instrumentation=instrum, budget=10000)

## check if method in registry

In [9]:
method = 'DE'

In [10]:
method = 'lalala'

In [11]:
if method not in list(registry.keys()):
    raise AssertionError('Unknown to Nevergrad optimizatino method: ' + method)

AssertionError: Unknown to Nevergrad optimizatino method: lalala

# Scipy-Optimize

In [12]:
from skopt import Optimizer as skoptOptimizer
from skopt.space import Real

In [13]:
instruments = []
for i, name in enumerate(parameter_names):
    # vars()[name] = Real(*bounds[i])
    instruments.append(Real(*bounds[i]))

In [14]:
instruments

[Real(low=-5, high=5, prior='uniform', transform='identity'),
 Real(low=0, high=10, prior='uniform', transform='identity')]

In [15]:
optimizer = skoptOptimizer(
    dimensions=instruments,
    random_state=1,
    base_estimator='gp'
)

## Check if candidates and parameters are even

In [16]:
from brian2tools import NevergradOptimizer, fit_traces_ask_tell

In [17]:
from brian2 import *
from numpy import zeros

In [18]:
# create input and output
input_traces = zeros((10,5))*volt
for i in range(5):
    input_traces[5:,i]=i*10*mV

output_traces = 10*nS*input_traces

In [19]:
model = Equations('''
    I = g*(v-E) : amp
    g : siemens (constant)
    E : volt (constant)
    ''')

In [20]:
n_opt = NevergradOptimizer(method='DE', parameter_names={'E', 'g'},
                           bounds=[[-5, 5], [0, 10]])

In [21]:
parameters = n_opt.ask(10)
parameters

[[0.2816587970965447, 7.86616814561799],
 [2.529812406166184, 2.9432813205058963],
 [1.9297235704178888, 4.8603585172650545],
 [-1.2906133680082756, 5.302380037136441],
 [-0.5920416354513165, 5.659383441096528],
 [-1.4233854457961437, 2.448504740684975],
 [0.9081871905924795, 2.1256821351473536],
 [-2.9572125808560403, 4.6996885821998235],
 [-3.780492608773249, 2.5241966814470493],
 [-3.1528971830826285, 5.117305807829553]]

In [22]:
candidates = n_opt.candidates
candidates

[Candidate(args=(0.2816587970965447, 7.86616814561799), kwargs={}, data=[0.08871739 1.26128016]),
 Candidate(args=(2.529812406166184, 2.9432813205058963), kwargs={}, data=[ 1.01890934 -0.75412711]),
 Candidate(args=(1.9297235704178888, 4.8603585172650545), kwargs={}, data=[ 0.69333759 -0.04389783]),
 Candidate(args=(-1.2906133680082756, 5.302380037136441), kwargs={}, data=[-0.42924198  0.09528228]),
 Candidate(args=(-0.5920416354513165, 5.659383441096528), kwargs={}, data=[-0.18817026  0.21016625]),
 Candidate(args=(-1.4233854457961437, 2.448504740684975), kwargs={}, data=[-0.47956913 -1.03289039]),
 Candidate(args=(0.9081871905924795, 2.1256821351473536), kwargs={}, data=[ 0.29331813 -1.26793498]),
 Candidate(args=(-2.9572125808560403, 4.6996885821998235), kwargs={}, data=[-1.33817935 -0.09462654]),
 Candidate(args=(-3.780492608773249, 2.5241966814470493), kwargs={}, data=[-2.48117745 -0.98491119]),
 Candidate(args=(-3.1528971830826285, 5.117305807829553), kwargs={}, data=[-1.52537827

In [26]:
import functools 

In [27]:
# initializing lists  
test_list1 = [1, 2, 4, 3, 5] 
test_list2 = [1, 2, 4, 3, 5]

In [28]:
if functools.reduce(lambda i, j : i and j, map(lambda m, k: m == k, test_list1, test_list2), True) :  
    print ("The lists are identical") 
else : 
    print ("The lists are not identical") 

The lists are identical


In [31]:
if functools.reduce(lambda i, j : i and j, map(lambda c, p: list(c.args) == p, candidates, parameters), True) :  
    print ("The lists are identical") 
else : 
    print ("The lists are not identical") 

The lists are identical


In [24]:
# pass parameters to the NeuronGroup
errors = fit_traces_ask_tell(model = model, input_var = 'v', output_var = 'I',\
                            input = input_traces, output = output_traces, dt = 0.1*ms,
                            g = [1*nS, 30*nS], E = [-20*mV,100*mV], update=parameters)

INFO       No numerical integration method specified, using method 'linear' (took 0.01s). [brian2.stateupdaters.base.method_choice]


bounds [[1. * nsiemens, 30. * nsiemens], [-20. * mvolt, 100. * mvolt]]


In [25]:
# give information to the optimizer
n_opt.tell(parameters, errors)
ans = n_opt.recommend()

[0.2816587970965447, 7.86616814561799] [0.2816587970965447, 7.86616814561799] True
[2.529812406166184, 2.9432813205058963] [2.529812406166184, 2.9432813205058963] True
[1.9297235704178888, 4.8603585172650545] [1.9297235704178888, 4.8603585172650545] True
[-1.2906133680082756, 5.302380037136441] [-1.2906133680082756, 5.302380037136441] True
[-0.5920416354513165, 5.659383441096528] [-0.5920416354513165, 5.659383441096528] True
[-1.4233854457961437, 2.448504740684975] [-1.4233854457961437, 2.448504740684975] True
[0.9081871905924795, 2.1256821351473536] [0.9081871905924795, 2.1256821351473536] True
[-2.9572125808560403, 4.6996885821998235] [-2.9572125808560403, 4.6996885821998235] True
[-3.780492608773249, 2.5241966814470493] [-3.780492608773249, 2.5241966814470493] True
[-3.1528971830826285, 5.117305807829553] [-3.1528971830826285, 5.117305807829553] True


AssertionError: Parameters and Candidates do not have identical values

In [ ]:
ans.args

In [ ]:
cand = [list(v.args) for v in candidates]
if np.all(params == cand):
    print('ok')

if np.all(params == [list(v.args) for v in candidates]):
    print('ok')